In [1]:
import json
import requests

In [2]:
add_ons = {}
errors = []

In [3]:
nhl_dict = requests.get('http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/teams')
nhl_dict = nhl_dict.json()

mlb_dict = requests.get('http://site.api.espn.com/apis/site/v2/sports/baseball/mlb/teams')
mlb_dict = mlb_dict.json()

nfl_dict = requests.get('http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/franchises?limit=50')
nfl_dict = nfl_dict.json()

nba_dict = requests.get('http://site.api.espn.com/apis/site/v2/sports/basketball/nba/teams')
nba_dict = nba_dict.json()



In [4]:
errors = []
team_links = nfl_dict['items']
for team in team_links:
    team_info = requests.get(team['$ref'])
    team_info = team_info.json()
    teamLink = requests.get(team_info['team']['$ref'])
    teamLink = teamLink.json()
    name = teamLink['displayName']
    logo = teamLink['logos'][0]['href']
    color = teamLink['color']
    alternateColor = teamLink['alternateColor']
    try:
        add_ons.update({
            name: {
                'logo': logo,
                'color': color,
                'alternateColor': alternateColor,
                'teamLink': team_info['team']['$ref'],
                'venue': {
                    'name': teamLink['venue']['fullName'],
                    'capacity': teamLink['venue']['capacity'],
                    'link': teamLink['venue']['$ref']
                }
            }})
    except Exception as e:
        errors.append({'error': e, 'team_info': team_info, 'team_link': teamLink})
    
    


In [5]:
errors

[]

In [6]:
team_number = nhl_dict['sports'][0]['leagues'][0]['teams'][0]['team']['id']
team_info = requests.get(f'http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/teams/{team_number}')
team_info = team_info.json()

In [7]:
team_info.keys()

dict_keys(['team'])

In [8]:
team_info['team']['franchise']['venue']['capacity']

17174

In [9]:
errors = []
for x in nhl_dict['sports'][0]['leagues'][0]['teams']:
    team_number = x['team']['id']
    team_link = f'http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/teams/{team_number}'
    team_info = requests.get(team_link)
    team_info = team_info.json()
    
    
    name = team_info['team']['name']
    logo = team_info['team']['logos'][0]['href']
    color = team_info['team']['color']
    alternateColor = team_info['team']['alternateColor']
    teamLink = team_link
    try:
        add_ons.update({
            name: {
                'logo': logo,
                'color': color,
                'alternateColor': alternateColor,
                'teamLink': team_link,
                'venue': {
                    'name': team_info['team']['franchise']['venue']['fullName'],
                    'capacity': team_info['team']['franchise']['venue']['capacity'],
                    'link': team_info['team']['franchise']['venue']['$ref']
                }
            }})
    except Exception as e:
        errors.append({'error': e, 'team_info': team_info, 'team_link': teamLink})

In [10]:
errors

[]

In [11]:
errors = []
for x in nba_dict['sports'][0]['leagues'][0]['teams']:
    name = x['team']['displayName']
    logo = x['team']['logos'][0]['href']
    color = x['team']['color']
    alternateColor = x['team']['alternateColor']
    try:
        add_ons.update({
            name: {
                'logo': logo,
                'color': color,
                'alternateColor': alternateColor,
                'teamLink': x['team']['links'][0]['href']
            }})
    except Exception as e:
        errors.append({'error': e, 'team_info': x})

In [12]:
errors

[]

In [13]:
errors = []
for x in mlb_dict['sports'][0]['leagues'][0]['teams']:
    name = x['team']['displayName']
    logo = x['team']['logos'][0]['href']
    color = x['team']['color']
    alternateColor = x['team']['alternateColor']
    try:
        add_ons.update({
            name: {
                'logo': logo,
                'color': color,
                'alternateColor': alternateColor,
                'teamLink': x['team']['links'][0]['href']
            }})
    except Exception as e:
        errors.append({'error': e, 'teamLink': teamLink})

In [14]:
errors

[]

In [15]:
f = open('teams.json')
data = json.load(f)
t = open('timezones.json')
timezones = json.load(t)

f = open('metro_area_venues.json')
stadiums = json.load(f)


In [16]:
sorted(list(data.keys()))

['MLB', 'MLS', 'NBA', 'NFL', 'NHL']

In [17]:
data['MLB'][0]

{'teamName': 'Arizona Diamondbacks',
 'city': 'Phoenix',
 'state': 'Arizona',
 'metroArea': 'Phoenix'}

In [18]:
clean_data = {}
all_teams_info = {}
for sport in data:
    for team in data[sport]:
        teamName = team['teamName']
        city = team['city']
        state = team['state']
        metroArea = team['metroArea']
        timeZone = timezones[metroArea]
        team.update({'league':sport})
        team.update({'timeZone':timeZone})
        if sport != 'MLS':
            try:
                team.update({'logo': add_ons[teamName]['logo']})
            except:
                team.update({'logo': ''})
            try:
                team.update({'color': add_ons[teamName]['color']})
            except:
                team.update({'color': ''})
            try:
                team.update({'alternateColor': add_ons[teamName]['alternateColor']})
            except:
                team.update({'alternateColor': ''})
            try:
                team.update({'teamLink': add_ons[teamName]['teamLink']})
            except:
                team.update({'teamLink': ''})
        if metroArea not in clean_data:
            clean_data.update({metroArea: [team]})
        else:
            area = clean_data[metroArea]
            area.append(team)
            clean_data.update({metroArea: area})
        
        all_teams_info.update({teamName: team})


In [19]:
final_data = []
number = 0
for x in clean_data:
    dict = {
        'id': number,
        'metroArea': x,
            'teams': clean_data[x]
           }
    final_data.append(dict)
    number += 1

In [20]:
def get_stadiums(metro_area, stadiums):
    for x in stadiums:
        if stadiums[x]['metro_area'] == metro_area:            
            return stadiums[x]['stadiums']
    print(f'Could not find {metro_area}')
    return []

In [21]:
final_data = sorted(final_data, key=lambda d: d['metroArea'])

In [22]:
for_real_final_data = []

In [23]:
for x in final_data:
    metro_area = x['metroArea']
    metro_area_stadiums = get_stadiums(metro_area, stadiums) 
    x.update({'stadiums' : metro_area_stadiums})
    
    

In [24]:
json_object = json.dumps(final_data, indent=4)
with open("metro_area_teamsnew.json", "w") as outfile:
    outfile.write(json_object)

In [25]:
len(final_data)

50

In [26]:
with open('MetroTeamsNewNew.js', 'w') as f:
    f.write(f'export const TeamsCatalog = {final_data}')
    f.close()

In [27]:
with open('AllTeamsInfoNew.js', 'w') as f:
    f.write(f'export const AllTeams = {all_teams_info}')
    f.close()